In [1]:
!pip install gdown

In [2]:
!gdown --id 1Mn34S75zJpvPMyk9ZfA5kXoUs5skhULD
# !gdown --id 1-5bdF9W6Ce9AI6YTj0SK8DmVjKFrN94H # phobert_2_task_att
# !gdown --id 1-00BXOpSHxAm_HUW7gbLWGuoE0N7L7h8 #phobert_2_task_att_v3

In [3]:
!pip install transformers
!pip install tokenizers
!pip install datasets
!pip install sacrebleu
!pip install sadice
# !pip install torchgeometry
!pip install sentencepiece
!pip install transformers sentencepiece vncorenlp

In [4]:
import transformers
import pandas as pd
from datasets import load_dataset, load_metric
import numpy as np
import pandas as pd
import pyarrow.parquet as pq
import pyarrow as pa
import time
import timeit
from tqdm.notebook import trange, tqdm
from time import sleep

import nltk
import sacrebleu
import torch
import torch.nn as nn

from sadice import SelfAdjDiceLoss

# Split data

In [5]:
import json

path_memory = "./memory_edges_3.json"
# path_memory = "../input/model-absa/memory_edges_3.json"

# Opening JSON file
with open(path_memory, 'rb') as openfile:
    # Reading from json file
    ls_data_valid = json.load(openfile)
print(len(ls_data_valid))
text2edge, ls_text, ls_asp, ls_sen = ls_data_valid.values()

In [6]:
from sklearn.model_selection import train_test_split
train_texts, val_texts, train_tags, val_tags, train_senti, val_senti = train_test_split(ls_text[:], ls_asp[:], ls_sen[:], test_size=.2)
len(train_texts), len(val_texts)

# Pretrain phoBERT

In [7]:
# from transformers import AutoModel, AutoTokenizer
# from transformers import BartTokenizerFast, BartphoTokenizer
# tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-large", do_lower_case=True,\
#                                              clean_text=True,handle_chinese_chars=True)
# model_phoBert = AutoModel.from_pretrained("vinai/phobert-large", )

# Pretrain BARTpho

In [8]:
# from transformers import AutoModel, AutoTokenizer
# from transformers import BartTokenizerFast, BartphoTokenizer
# tokenizer = AutoTokenizer.from_pretrained("vinai/bartpho-word", do_lower_case=True,\
#                                              clean_text=True,handle_chinese_chars=True)
# model_bart = AutoModel.from_pretrained("vinai/bartpho-word", )

In [9]:
# a = 'giao hàng nhanh nhưng giá cao'
# tokens = tokenizer(a, return_tensors='pt')
# feature = model_bart(tokens['input_ids'], tokens['attention_mask'])
# feature

# function for feature

In [18]:
def convert_ids_to_string(ids, tokenizer):
    ls = []
    for id in ids:
        # s = tokenizer.convert_tokens_to_string(tokenizer._convert_id_to_token(int(id)))
        s = tokenizer.convert_ids_to_tokens(int(id))
        ls.append(s)
    return ls

def pair_labels(texts, asp, senti, tokenizer):
    dict_words = []
    for i,word in enumerate(texts):
        ls_id = []
        tokens = tokenizer.tokenize(word)
        for token in tokens:
            # id = tokenizer._convert_token_to_id(token)
            id = tokenizer.convert_tokens_to_ids(token)
            ls_id.append(id)
        dict_words.append([word,ls_id,asp[i],senti[i]])
    return dict_words

def balance_labels(texts, asp_label, senti_label):
    Words = pair_labels(texts, asp_label, senti_label)
    asp_label_new = []
    senti_label_new = []
    texts_new = []
    for id in Words:
        for i in id[1]:
            texts_new.append(tokenizer.convert_ids_to_tokens(i))
            asp_label_new.append(id[2])
            senti_label_new.append(id[3])
    return texts_new,asp_label_new,senti_label_new


def pair_labels_v2(texts, labels):
    dict_words = []
    for i,word in enumerate(texts):
        ls_id = []
        tokens = tokenizer.tokenize(word)
        for token in tokens:
            # id = tokenizer._convert_token_to_id(token)
            id = tokenizer.convert_tokens_to_ids(token)
            ls_id.append(id)
        dict_words.append([word,ls_id,labels[i]])
    return dict_words    


def balance_labels_v2(texts, labels):
    Words = pair_labels_v2(texts, labels)
    label = []
    texts_new = []
    for id in Words:
        for i in id[1]:
            texts_new.append(tokenizer.convert_ids_to_tokens(i))
            label.append(id[2])
            
    return texts_new, label

In [19]:
def get_feature_asp(text, tags, senti):
    """
    Extract aspect and sentiment for aspect
        text: separated words in a sentence (list)
        tags: label aspects of a sentence (list)
        senti: label sentiments of a sentence (list)
    """
    memo = {'text':[], 'asp_senti':[], 'label_asp':[]}
    for j,(t,ap,sen) in enumerate(zip(text,tags,senti)):
        
        temp = []
        if ap == 'B-asp':

            temp.append([text[j], senti[j]])
            if j+1 < len(tags):
                try:
                    while tags[j+1] == 'I-asp':
                        temp.append([text[j+1], senti[j+1]])
                        j += 1
                except IndexError:
                    continue

            # print(temp)    
            
            # Key: text, Value: [asp, senti]             
            if len(temp) > 1:
                asp = []
                for ap in temp:
                    asp.append(ap[0])
                aspect = ' '.join(asp)
                sen = ap[1]
            else:
                aspect = temp[0][0]
                sen = temp[0][1]

            if len(temp) < 1:
                continue
            memo['asp_senti'].append([aspect,sen])
        memo['text'] = ' '.join(text)
        memo['label_asp'] = tags
        memo['label_senti'] = senti

    return memo


    
i = 5000
get_feature_asp(ls_text[i], ls_asp[i], ls_sen[i])

# Encode label & WNUT dataset

In [12]:
# encoding label
labels2id = {'O': 0, 'B-0': 1, 'B-1': 2, 'B-2': 3, 'I-0': 4, 'I-1': 5, 'I-2': 6}

tag2id = {'O': 0, 'B-asp': 1, 'I-asp': 2 }
id2tag = {id: tag for tag, id in tag2id.items()}
print(id2tag)
sen2id = {'Neg': 0, 'Pos': 1, 'Neu': 2, '-1':3}
id2sen = {id: sen for sen, id in sen2id.items()}
print(id2sen)

sen2id_vn = {'Xấu': 0, 'Tốt': 1, 'Bình thường': 2}
id2sen_vn = {id: sen for sen, id in sen2id_vn.items()}
print(id2sen_vn)

sen2id_2 = {'0': 0, '1': 1, '2': 2, '-1':3}
id2sen_2 = {id: sen for sen, id in sen2id.items()}
print(id2sen)

In [16]:
def encode_tags(texts, aspect, senti, encodings, tokenizer):
    encoded_label_asp = []
    encoded_label_senti = []
    for text, asp, sent, doc_ids in tqdm(zip(texts, aspect, senti, encodings['input_ids']), desc = 'encoding loop'):
        # create an empty array of -100
        ids = list(doc_ids)

        doc_enc_label_asp = np.ones(len(ids),dtype=int) * -100
        doc_enc_label_senti = np.ones(len(ids),dtype=int) * -100
        Words = pair_labels(text, asp, sent, tokenizer)
        
        # indices = [j for j in range(ids.index(0), ids.index(2))]
        for id in Words:
            for i in id[1]:
                # find all index of id
                # print(ids)
                indices = [index for index, element in enumerate(ids) if element == i ]
                for j in indices:
                    doc_enc_label_asp[j] = tag2id[id[2]]  
                    doc_enc_label_senti[j] = sen2id_2[id[3]]
                # <s> and </s>
                indices = [index for index, element in enumerate(ids) if element == 0 or element == 2]
                for j in indices:
                    doc_enc_label_asp[j] =  0
                    doc_enc_label_senti[j] = -1
        encoded_label_asp.append(doc_enc_label_asp)
        encoded_label_senti.append(doc_enc_label_senti)

    return encoded_label_asp, encoded_label_senti


# # train_labels_asp, train_labels_sen ,train_encodings = encode_tags(train_texts, train_labels, train_encodings)
# # val_labels_asp, val_labels_sen ,val_encodings = encode_tags(val_texts, val_labels, val_encodings)

In [20]:
import torch
from transformers import AutoModel, AutoTokenizer
from transformers import BartTokenizerFast, BartphoTokenizer

class WNUTDataset_2(torch.utils.data.Dataset):
    def __init__(self, ls_texts, labels_asp, labels_sen, device, tokenizer = "phobert"):
        """
            ls_texts: separated words of sentences (list of sentences)
            labels_asp: label aspects (list of sentences)
            labels_sen: label sentiments (list of sentences)
            device: "cpu" or "cuda"
            tokenizer: tokenize by "phobert" or "bartpho" (string)
        """
        if tokenizer.lower() == "phobert":
            tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-large", do_lower_case=True,\
                                     clean_text=True,handle_chinese_chars=True)
        else:
            tokenizer = AutoTokenizer.from_pretrained("vinai/bartpho-word", do_lower_case=True,\
                                     clean_text=True,handle_chinese_chars=True)

        self.ls_texts = ls_texts
        self.labels_asp = labels_asp
        self.labels_sen = labels_sen

        self.features =  [get_feature_asp(text, tag, sen) for text,tag,sen in zip(ls_texts[:], labels_asp[:], labels_sen[:]) ]

        texts = [feature['text'] for feature in self.features]
        
        self.encoding = tokenizer(texts, return_tensors='pt', padding=True, truncation=True, max_length=250)
        
        self.texts = texts
        self.encode_asp, self.encode_senti = encode_tags(ls_texts[:], labels_asp[:], labels_sen[:], self.encoding, tokenizer) 
        self.device = device
    def __getitem__(self, idx):
        
        item = {key: torch.tensor(val[idx]).to(self.device, dtype = torch.long) for key, val in self.encoding.items()}
        item['ls_texts'] = self.ls_texts[idx]
        item['encode_asp'] = torch.tensor(self.encode_asp[idx]).to(self.device, dtype = torch.long)
        item['encode_senti'] = torch.tensor(self.encode_senti[idx]).to(self.device, dtype = torch.long)
        item['labels_asp'] = self.labels_asp[idx]
        item['labels_sen'] = self.labels_sen[idx]
        item['features'] = self.features[idx]
        item['texts'] = self.texts[idx]
        return item

    def __len__2(self):
        return ["2 labels", len(self.labels_asp), len(self.labels_sen)]
    
    def __len__(self):
        return len(self.labels_asp)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_dataset_2 = WNUTDataset_2(train_texts, train_tags, train_senti, device, tokenizer = 'phobert')
val_dataset_2 = WNUTDataset_2(val_texts, val_tags, val_senti, device, tokenizer = 'phobert')

In [21]:
# DATA LOADER
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

var = 1


loaders = {
    'train' : DataLoader(train_dataset_2,batch_size = var),
    'test'  : DataLoader(val_dataset_2,batch_size = var),
}

# Model senti

In [22]:
import torch.nn.functional as F
def adjacency_matrix(edges, attention_shape):
    matrix = torch.zeros(attention_shape[1:])
    # print(graph)
    # print('shape:',len(matrix))
    for [i,j] in edges:
        # print('shape:',len(matrix),len(matrix[key]))
#             print(key,i)
#         if i > attention.shape[1] or j > attention.shape[1]:
#             continue
        try:
            matrix[i,j] = 1
        except IndexError:
            continue
    return matrix
def scaled_dot_product(q, k, v,edge,tau = 0.9):
    devive = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    d_k = q.size()[-1]
    attn_logits = torch.matmul(q, k.transpose(-2, -1))
#     attn_logits = F.softmax(attn_logits/tau, dim=-1)
    attn_logits = attn_logits / np.sqrt(d_k)
#     print('before softmax: \n',attn_logits)
#     print(attn_logits.shape)
    adj_matrix = adjacency_matrix(edge, attn_logits.shape)
#     print('adj: \n',adj_matrix)
#     attn_logits = torch.mul(adj_matrix.to('cuda'), attn_logits)
    
    attention = F.softmax(attn_logits/tau, dim=-1)
    attention = attention.add(adj_matrix.to(device))
    values = torch.matmul(attention, v)
#     print('final attention: \n',attention)
    return values, attention

# def scaled_dot_product(q, k, v,tau = 0.9):
#     devive = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#     d_k = q.size()[-1]
#     attn_logits = torch.matmul(q, k.transpose(-2, -1))
#     attn_logits = attn_logits / np.sqrt(d_k)
#     attention = F.softmax(attn_logits/tau, dim=-1)
#     values = torch.matmul(attention, v)
#     return values, attention

In [23]:
## Test thử
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

class ABSA(nn.Module):
    def __init__(self, pre_model = 'phobert'):
        '''
            pre_model: model pre-training "phobert" or "bartpho" (string)
        '''
        super().__init__()
        self.num_labels_token = 3
        self.num_labels_sen = 3
        

        
        ####### phoBERT #######
        if pre_model.lower() == 'phobert':
            self.tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-large", do_lower_case=True,\
                                                 clean_text=True,handle_chinese_chars=True)
            self.model = AutoModel.from_pretrained("vinai/phobert-large")
        elif pre_model.lower() == 'bartpho':
        ####### BARTpho ######
            self.tokenizer = AutoTokenizer.from_pretrained("vinai/bartpho-word", do_lower_case=True,\
                                                 clean_text=True,handle_chinese_chars=True)

            self.model = AutoModel.from_pretrained("vinai/bartpho-word", )
            
        #======== freeze encoder ==========
#         freeze_module = self.bart.encoder.layers[:]
#         for param in freeze_module.parameters():
#             param.requires_grad = False
            
        #========= freeze all ========
        # for param in self.bert.parameters():
        #     param.requires_grad = False
    
        # ======== Layer classifier ===========
        self.classifier_sen = nn.Linear(1024, 3)
#         self.classifier_asp = nn.Linear(1024, 3)
        self.classifier_asp = nn.Linear(1024, 3)
        
        # ======= layer GCN for aspect =======
        self.linearasp_1 = nn.Linear(1024,1024)
        self.linearasp_2 = nn.Linear(1024,1024)
        self.linearasp_3 = nn.Linear(1024,1024)
        
        # ======== layer GCN for sentiment ======
        self.linearsen_1 = nn.Linear(1024,1024)
        self.linearsen_2 = nn.Linear(1024,1024)
        self.linearsen_3 = nn.Linear(1024,1024)
    #################################################
    def forward_GCN_2task(self,
                   text=None, 
                    input_ids = None, 
                    attention_mask = None, 
                    feature = None,
                    encode_asp = None,
                    encode_senti = None,
                    device = 'cuda',
                    label_aspect = None, label_senti = None):
        
        embedding_1 = self.model(input_ids, attention_mask, output_hidden_states=True)
        
        Q_asp = self.linearasp_1(embedding_1['last_hidden_state'])
        K_asp = self.linearasp_2(embedding_1['last_hidden_state'])
        V_asp = self.linearasp_3(embedding_1['last_hidden_state'])
        
        Values_asp, att_weights_asp = scaled_dot_product(Q_asp,K_asp,V_asp, text2edge[text])
        logits_token = self.classifier_asp(Values_asp)
        
        for label_aspect,label_senti in feature['asp_senti']:
            text_new = text+" </s> "+label_aspect
            encoding_2 = self.tokenizer(text_new, return_tensors='pt', padding=True, truncation=True, max_length=250).to(device)
#             ids = torch.cat((input_ids, encoding_2['input_ids']), dim=1)
#             mask = torch.cat((attention_mask, encoding_2['attention_mask']),dim=1)
            embedding_2 = self.model(encoding_2['input_ids'],  encoding_2['attention_mask'], output_hidden_states=True)
            Q_sen = self.linearsen_1(embedding_2['last_hidden_state'])
            K_sen = self.linearsen_2(embedding_2['last_hidden_state'])
            V_sen = self.linearsen_3(embedding_2['last_hidden_state'])
            Values_sen, att_weights_sen = scaled_dot_product(Q_sen,K_sen,V_sen, text2edge[text])
            output_2 = mean_pooling(Values_sen,encoding_2['attention_mask']).view(-1)
            logits_sen = self.classifier_sen(output_2)
            
        
    def forward_task2(self,\
                      text = None,
                      input_ids = None,\
                      attention_mask = None,\
                      feature = None,\
                      label_aspect = None,\
                      label_senti = None,\
                      device = None):
        # Sentence + aspect => sentiment
        
        
        ##  C1 concat sau khi embedding câu gốc và aspect ===================================================  
#         embedding_text = self.model(input_ids,  attention_mask, output_hidden_states=True)
#         encoding_2 = self.tokenizer(label_aspect, return_tensors='pt', padding=True, truncation=True, max_length=250).to(device)
        
#         embedding_aspect = self.model(encoding_2['input_ids'],  encoding_2['attention_mask'], output_hidden_states=True)
        
#         embedding_concat = torch.cat((embedding_text['last_hidden_state'], embedding_aspect['last_hidden_state']),dim=1)
#         mask_concat = torch.cat((attention_mask, encoding_2['attention_mask']),dim=1)
#         output = mean_pooling(embedding_concat,mask_concat).view(-1)
#         logits_sen = self.classifier_sen(output)

        ##### Compine GCN #####
#         embedding_text = self.model(input_ids,  attention_mask, output_hidden_states=True)
#         encoding_2 = self.tokenizer(label_aspect, return_tensors='pt', padding=True, truncation=True, max_length=250).to(device)
#         embedding_aspect = self.model(encoding_2['input_ids'],  encoding_2['attention_mask'], output_hidden_states=True)
#         embedding_concat = torch.cat((embedding_text['last_hidden_state'], embedding_aspect['last_hidden_state']),dim=1)
#         mask_concat = torch.cat((attention_mask, encoding_2['attention_mask']),dim=1)
#         Q_sen = self.linearsen_1(embedding_concat)
#         K_sen = self.linearsen_2(embedding_concat)
#         V_sen = self.linearsen_3(embedding_concat)
#         Values_sen, att_weights_sen = scaled_dot_product(Q_sen,K_sen,V_sen, text2edge[text])
#         output = mean_pooling(Values_sen,mask_concat).view(-1)
#         logits_sen = self.classifier_sen(output)
    
        ## C2  Nối câu gốc với aspect ========================================================================
        text_new = text+" </s> "+label_aspect
        encoding_2 = self.tokenizer(text_new, return_tensors='pt', padding=True, truncation=True, max_length=250).to(device)
        embedding_2 = self.model(encoding_2['input_ids'],  encoding_2['attention_mask'], output_hidden_states=True)
        
#         output = mean_pooling(embedding_2['last_hidden_state'],encoding_2['attention_mask']).view(-1)
#         logits_sen = self.classifier_sen(output)
        
        ##### Compine GCN #####
        Q_sen = self.linearsen_1(embedding_2['last_hidden_state'])
        K_sen = self.linearsen_2(embedding_2['last_hidden_state'])
        V_sen = self.linearsen_3(embedding_2['last_hidden_state'])
        Values_sen, att_weights_sen = scaled_dot_product(Q_sen,K_sen,V_sen, text2edge[text])
#         Values_sen, att_weights_sen = scaled_dot_product(Q_sen,K_sen,V_sen)
        output = mean_pooling(Values_sen,encoding_2['attention_mask']).view(-1)
        logits_sen = self.classifier_sen(output)
        
        loss_sen = None
        if label_senti is not None:
            loss_fct_sen = nn.CrossEntropyLoss()
#             weight_sen = torch.tensor([2., 1., 3.]).to(device)
#             loss_fct_sen = nn.CrossEntropyLoss(weight = weight_sen)
            loss_sen = loss_fct_sen(logits_sen, torch.tensor(abs(int(label_senti)), dtype = torch.long).to(device))
         
        return logits_sen, loss_sen
    ####################################################
    def forward_task1(self, 
                text=None, 
                input_ids = None, 
                attention_mask = None, 
                feature = None,
                encode_asp = None,
                encode_senti = None,
                device = None):
        # Token classification

#         embedding_1 = self.model(input_ids, attention_mask, output_hidden_states=True)
#         logits_token = self.classifier_asp(embedding_1['last_hidden_state'])
        
        # Compine GCN
        embedding_1 = self.model(input_ids, attention_mask, output_hidden_states=True)
        Q_asp = self.linearasp_1(embedding_1['last_hidden_state'])
        K_asp = self.linearasp_2(embedding_1['last_hidden_state'])
        V_asp = self.linearasp_3(embedding_1['last_hidden_state'])
        
        Values_asp, att_weights_asp = scaled_dot_product(Q_asp,K_asp,V_asp, text2edge[text])
#         Values_asp, att_weights_asp = scaled_dot_product(Q_asp,K_asp,V_asp)
        logits_token = self.classifier_asp(Values_asp)
        
        loss_asp = None
        if (encode_asp is not None):
            weight_token = torch.tensor([1., 10., 10.]).to(device)
            loss_fct_token = nn.CrossEntropyLoss(weight = weight_token)
            
#             loss_fct_token = nn.CrossEntropyLoss()

            # Only keep active parts of the loss
            if attention_mask is not None:
                active_loss = attention_mask.view(-1) == 1

                active_logits_token = logits_token.view(-1, self.num_labels_token)

                active_labels_token = torch.where(
                    active_loss, encode_asp.view(-1), torch.tensor(loss_fct_token.ignore_index).type_as(encode_asp)
                )

                loss_asp = loss_fct_token(active_logits_token, active_labels_token) 
            else:
                loss_asp = loss_fct_token(logits_token.view(-1, self.num_labels_token), encode_asp.view(-1))
        
        return logits_token, loss_asp

     #########################################   
    def forward_2_task(self,
                text=None, 
                input_ids = None, 
                attention_mask = None, 
                feature = None,
                encode_asp = None,
                encode_senti = None,
                device = None,
                label_aspect = None, label_senti = None,):
        
        logits_token, loss_asp = self.forward_task1(
                                        text= text,\
                                        input_ids = input_ids,\
                                        attention_mask = attention_mask,\
                                        feature = feature,\
                                        encode_asp = encode_asp,\
                                        encode_senti = encode_senti,\
                                        device = device)
         
        ls_logits_sen = []
        loss_tam = 0
        loss= None
        for label_aspect,label_senti in feature['asp_senti']:
            # print(aspect,senti)
            logits_sen, loss_sen = self.forward_task2(
                                                text = text,\
                                                input_ids = input_ids,\
                                                attention_mask = attention_mask,\
                                                feature = feature,\
                                                device = device,\
                                                label_aspect = label_aspect[0], label_senti = label_senti[0])                    
            loss_tam += loss_sen
            ls_logits_sen.append(logits_sen)
        if encode_asp is not None:
            loss = loss_asp + loss_tam
        
        
        return logits_token, ls_logits_sen, loss
    #################################################
    def forward_task3(self, 
                text=None, 
                input_ids = None, 
                attention_mask = None, 
                feature = None,
                encode_asp = None,
                encode_senti = None,
                device = 'cuda'):
        
        ls_logits_asp = []
        loss_tam = 0
        # feature['asp_senti']: [aspect, senti]
        for aspect,senti in feature['asp_senti']:
            encoding_3 = tokenizer(id2sen_vn[int(senti[0])], return_tensors='pt', padding=True, truncation=True, max_length=250).to(device)
            ids = torch.cat((input_ids, encoding_3['input_ids']), dim=1)
            mask = torch.cat((attention_mask, encoding_3['attention_mask']),dim=1)
            embedding_3 = self.bart(ids, mask, output_hidden_states=True)
            # output_3 = mean_pooling(embedding_3, encoding_3["attention_mask"]).view(-1)
            logits_token = self.classifier_asp(embedding_3['last_hidden_state'])
            ls_logits_asp.append(logits_token)
            encode_asp_label = encode_asp

            if (encode_asp_label is not None):
                temp = encode_asp_label[0]
                ls_temp = []
                for i in range(0, encoding_3['input_ids'].shape[1]):
                    ls_temp.append(0)
                temp = torch.cat((temp.to('cpu'),torch.tensor(ls_temp)))
                # print(temp.shape, torch.reshape(temp, (1,len(temp))).shape)
                encode_asp_label =  torch.reshape(temp, (1,len(temp))).to(device)
        
            loss_fct_token = nn.CrossEntropyLoss()

            # Only keep active parts of the loss
            if mask is not None:
                active_loss = mask.view(-1) == 1

                active_logits_token = logits_token.view(-1, self.num_labels_token)

                active_labels_token = torch.where(
                    active_loss, encode_asp_label.view(-1), torch.tensor(loss_fct_token.ignore_index).type_as(encode_asp_label)
                )

                loss = loss_fct_token(active_logits_token, active_labels_token) 
            else:
                loss = loss_fct_token(logits_token.view(-1, self.num_labels_token), encode_asp_label.view(-1))
            loss_tam += loss
        loss_asp = loss_tam

        return logits_token, loss_asp
    ###############################################
    def forward(self, 
                text=None, 
                input_ids = None, 
                attention_mask = None, 
                feature = None,
                encode_asp = None,
                encode_senti = None,
                device = 'cuda',
                label_aspect = None, label_senti = None, 
                classes_senti_vn = id2sen_vn, 
                task_1 = False, task_2 = False, task_3 = False, Multi_task = False):      

        # =================================================
        if task_1:
            return self.forward_task1(
                                text= text,\
                                input_ids = input_ids,\
                                attention_mask = attention_mask,\
                                feature = feature,\
                                encode_asp = encode_asp,\
                                encode_senti = encode_senti,\
                                device = device)
            
        # =================================================    
        
        if task_2:
            return self.forward_task2(
                                    text = text,\
                                    input_ids = input_ids,\
                                    attention_mask = attention_mask,\
                                    feature = feature,\
                                    label_aspect = label_aspect,\
                                    label_senti = label_senti,\
                                    device = device,)

        # =================================================
        
        if task_3:
            return self.forward_task3(input_ids = input_ids,\
                                attention_mask = attention_mask,\
                                feature = feature,\
                                encode_asp = encode_asp,\
                                encode_senti = encode_senti,\
                                device = device)
        if Multi_task:
            return self.forward_2_task(
                            text=text, 
                            input_ids = input_ids, 
                            attention_mask = attention_mask, 
                            feature = feature,
                            encode_asp = encode_asp,
                            encode_senti = encode_senti,
                            device = device)
                    
        


## Train model

In [24]:
import shutil
def save_ckp(state, is_best, checkpoint_path, best_model_path):
    """
    state: checkpoint we want to save
    is_best: is this the best checkpoint; min validation loss
    checkpoint_path: path to save checkpoint
    best_model_path: path to save best model
    """
    f_path = checkpoint_path
    # save checkpoint data to the path given, checkpoint_path
    torch.save(state, f_path)
    # if it is a best model, min validation loss
    if is_best:
        best_fpath = best_model_path
        # copy that checkpoint file to best path given, best_model_path
        shutil.copyfile(f_path, best_fpath)

In [25]:
def train(start_epochs, n_epochs, valid_loss_min_input,\
          data_loaders, device, model, optimizer, checkpoint_path, best_model_path, task = 1):
    """
    Keyword arguments:
    start_epochs -- the real part (default 0.0)
    n_epochs -- the imaginary part (default 0.0)
    valid_loss_min_input
    loaders
    model
    optimizer
    criterion
    use_cuda
    checkpoint_path
    best_model_path
    task: 1,2,3 (AE, APC, Multi_task) (default: 1)
    returns trained model
    """
    # initialize tracker for minimum validation loss
    valid_loss_min = valid_loss_min_input 
    
    
    loss_tam2 = []
    ls_train_loss = []
    ls_eval_loss = []

    
    
    for epoch in range(start_epochs, n_epochs+1):
        # initialize variables to monitor training and validation loss
        # train_loss = 0.0
        # valid_loss = 0.0
        
        ###################
        # train the model #
        ###################
        print("\nStarting training...")
        train_start_time = timeit.default_timer()
        model.train()
        loss_tam = []
        for num_senten,data in enumerate(tqdm(data_loaders['train'], desc = 'training loop')):
            text = data['texts'][0]
            input_ids = data['input_ids']
            attention_mask = data['attention_mask']
            feature = data['features']
            encode_asp = data['encode_asp']

            ############ output model ############
            if task == 1:
                #================= aspect extraction =======================
                ### zero the parameter gradients        
                optimizer.zero_grad()
                logits_asp, loss_asp = model(text, input_ids, attention_mask,\
                                             feature, encode_asp,\
                                             device = device, task_1 = True)
                loss_tam.append(loss_asp.item())
                loss_asp.backward()
                optimizer.step()
                
            elif task == 2:
                #================= Sentiment for aspects =======================            
                for aspect,senti in feature['asp_senti']:
                    optimizer.zero_grad()
                    logits_sen, loss_sen = model(text, input_ids, attention_mask, \
                                                 label_aspect = aspect[0], label_senti = senti[0],\
                                                 device = device, task_2 = True)
                    loss_tam.append(loss_sen.item())
                    loss_sen.backward()
                    optimizer.step()
            else:
                # ================= Multi task =============================
                optimizer.zero_grad()
                logits_asp, logits_sen, loss = model(text, input_ids, attention_mask,\
                                                feature = feature, encode_asp = encode_asp,\
                                                device = device, Multi_task = True)
                loss_tam.append(loss.item())
                loss.backward()
                optimizer.step()
                
        ls_train_loss.append(np.mean(loss_tam))
                  
        ######################    
        # validate the model #
        ######################    
        
        print("\nStarting evaluation...")
        model.eval()
        with torch.no_grad():
            eval = []     
            for num_senten,data in enumerate(data_loaders['test']):
                text = data['texts'][0]
                input_ids = data['input_ids']
                attention_mask = data['attention_mask']
                feature = data['features']
                encode_asp = data['encode_asp']
                
                ############# output model ##############
                if task == 1:
                    #================= aspect extraction =======================
                    logits_asp, loss_asp = model(text, input_ids, attention_mask,\
                                                 feature, encode_asp,\
                                                 device = device, task_1 = True)
                    eval.append(loss_asp.item())
                elif task == 2:
                    #=========== sentiment for aspects =================
                    for aspect,senti in feature['asp_senti']:
                        logits_sen, loss_sen = model(text, input_ids, attention_mask, \
                                                     label_aspect = aspect[0], label_senti = senti[0],\
                                                     device = device, task_2 = True)


                        eval.append(loss_sen.item())
                else:
                    # ================= Multi task =============================
                    logits_asp,logits_sen, loss = model(text, input_ids, attention_mask,\
                                                feature = feature, encode_asp = encode_asp,\
                                                device = device, Multi_task = True)

                    eval.append(loss.item())
                    
        ls_eval_loss.append(np.mean(eval))         
        # print training/validation statistics
        print(f"Time train epoch {epoch}: {timeit.default_timer() - train_start_time }") 
        print('>>> Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
            epoch, 
            np.mean(loss_tam),
            np.mean(eval)
            ))        
        loss_tam = []
        ######################
        torch.cuda.empty_cache()

        train_loss = ls_train_loss[-1]
        valid_loss = ls_eval_loss[-1]
        # create checkpoint variable and add important data
        checkpoint = {
            'epoch': epoch + 1,
            'valid_loss_min': valid_loss,
            'state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict(),
        }
        
        # save checkpoint
        save_ckp(checkpoint, False, checkpoint_path, best_model_path)
        
        ## TODO: save the model if validation loss has decreased
        if valid_loss <= valid_loss_min:
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,valid_loss))
            # save checkpoint as best model
            save_ckp(checkpoint, True, checkpoint_path, best_model_path)
            valid_loss_min = valid_loss
        # Early stop train
        if epoch >= 2:
            if train_loss - ls_train_loss[-2] > 0 or valid_loss - ls_eval_loss[-2] > 0:
                return model,ls_train_loss, ls_eval_loss
            
    # return trained model
    return model,ls_train_loss, ls_eval_loss

In [26]:
devive = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = ABSA(pre_model='phobert')
model_parameters = filter(lambda p: p.requires_grad, model.parameters())
params = sum([np.prod(p.size()) for p in model_parameters])
print(params) # khoong freeze, train laij pretrained bert
model.to(device)

In [27]:
from datetime import date
name_model = 'ABSA'
today = date.today()
d1 = today.strftime("%d%m%y")
d1

In [28]:
LEARNING_RATE = 3e-7
weight_decay = 1e-4
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE, weight_decay= weight_decay)

In [29]:
EPOCHS = 1
path_checkpoint = f"/checkpoint_{d1}_ABSA_phoBert_2_task_new_att.pt"
path_bestmodel = f"./best_model_ABSA_phoBert_2_task_new_att.pt"
trained_model, train_loss, valid_loss = train(1, EPOCHS, np.Inf, loaders, device, model, optimizer,\
                                              path_checkpoint, path_bestmodel, task = 2)

In [30]:
import matplotlib.pyplot as plt
# %matplotlib inline

import seaborn as sns

# Use plot styling from seaborn.
sns.set(style='darkgrid')

# Increase the plot size and font size.
# sns.set(font_scale=1.5)
plt.rcParams["figure.figsize"] = (12,6)

# Plot the learning curve.
plt.plot(train_loss, 'b-o', label="training loss")
plt.plot(valid_loss, 'r-o', label="validation loss")

# Label the plot.
plt.title("Learning curve")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()

# plt.show()

# Load model

In [31]:
## keep on Train model
# EPOCHS = 8
# path_checkpoint = f"/checkpoint_{d1}_ABSA_2_task.pt"
# path_bestmodel = f"./best_model_ABSA_2_task.pt"
# trained_model, train_loss, valid_loss = train(epoch_recall, EPOCHS, loss_recall, loaders, device, model_recall, optimizer_recall,\
#                                               path_checkpoint, path_bestmodel)

In [32]:
torch.cuda.empty_cache()

In [33]:
# PATH = '../input/model-absa/best_model_ABSA_phoBert_sen_v2.pt'
PATH = './best_model_ABSA_phoBert_2_task_new_att.pt' 
model_recall = ABSA()
LEARNING_RATE = 3e-7
weight_decay= 1e-4
optimizer_recall = torch.optim.Adam(params =  model_recall.parameters(), lr=LEARNING_RATE,  weight_decay= weight_decay)
devive = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
checkpoint = torch.load(PATH, map_location=torch.device(device))
model_recall.load_state_dict(checkpoint['state_dict'])
optimizer_recall.load_state_dict(checkpoint['optimizer'])
epoch_recall = checkpoint['epoch']
loss_recall = checkpoint['valid_loss_min']
model_parameters = filter(lambda p: p.requires_grad, model_recall.parameters())
params = sum([np.prod(p.size()) for p in model_parameters])
print(params)
model_recall.to(device)
# model_recall.eval()

# Test model

In [34]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

def evaluate_model_both_task(model, loaders, device, tokenizer = 'phobert', ATE = False, APC = False, M_task = False,\
                                classes_asp = tag2id,\
                                classes_senti = sen2id_2):
    

    Result = {'acc_asp': None, 'acc_senti': None , 'F1_asp': None, 'F1_senti': None}
    eval_data = []
    n_test_correct, n_test_total = 0,0
    n_test_senti_correct, n_test_senti_total = 0,0
    n_test_asp_correct, n_test_asp_total = 0,0

    actual_test_label, model_label = [],[]
    total_labels_actual, total_labels_predict = [],[]
    total_asp_predict, total_senti_predict = [],[]
    total_asp_actual, total_senti_actual = [],[]
    Wrong_aspect, Wrong_tokens = [],[]
    model.eval()
    
    if tokenizer.lower() == "phobert":
        tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-large", do_lower_case=True,\
                                 clean_text=True,handle_chinese_chars=True)
    elif tokenizer.lower() == "bartpho":
        tokenizer = AutoTokenizer.from_pretrained("vinai/bartpho-word", do_lower_case=True,\
                                 clean_text=True,handle_chinese_chars=True)
        

 
    if isinstance(classes_asp, list) == False or isinstance(classes_senti, list) == False:
        classes_senti = list(classes_senti)
        classes_asp = list(classes_asp)
            
    for i,data in enumerate(tqdm(loaders['test'], desc='evaluation_model_both_task')):
        actual_test_senti_label,model_senti_label = [],[]
        actual_test_asp_label,model_asp_label = [],[]
        text = data['texts'][0]
        feature = data['features']  
        if APC:
#             if i == 100:
#                 break
            label_asp_senti = feature['asp_senti']
            print('>>> Sentiment for aspects')
            print('Text: ', text)

            for aspect,senti in label_asp_senti:
                paraphrase = tokenizer(text, return_tensors="pt").to(device)
                logits_sen, loss_sen = model(text, paraphrase['input_ids'], paraphrase['attention_mask'],\
                                             label_aspect = aspect[0], device = device, task_2 = True)
                ind = torch.argmax(logits_sen.clone().detach()).item()
                print('label: ', aspect[0],  senti[0])
                print('predict: ',aspect[0], ind)
                actual_test_senti_label.append(int(senti[0]))
                model_senti_label.append(int(ind))
            print('='*40)
        
        if ATE:
            
            paraphrase = tokenizer(text, return_tensors="pt", max_length = 250).to(device)
            label_asp = np.array(data['labels_asp']).flatten()
            size = len(paraphrase['input_ids'][0])
#             print('abc')
#             if i == 10:
#                 break
            logits_asp, loss_asp = model(text, paraphrase['input_ids'], paraphrase['attention_mask'],\
                                         feature, device = device, task_1 = True)

        
            n_test_total += (size-2)
            tokens = convert_ids_to_string(paraphrase['input_ids'][0], tokenizer)
            eval_data.append(tokens)

#             print(logits_asp[0].clone().detach())
            for ix,(u) in enumerate(tqdm(zip(logits_asp[0].clone().detach()), desc = f"{i}th_sentence")):
                try:
#                     print(u)
                    ind_u = torch.argmax(u[0].clone().detach()).item()
                    
                    if (classes_asp[ind_u] == label_asp[ix-1]): n_test_asp_correct += 1
                    model_asp_label.append(classes_asp[ind_u])
                    actual_test_asp_label.append(label_asp[ix-1])


                except:
#                     print('except')
#                     ind_u = torch.argmax(u[0].clone().detach()).item()
                    # ind_v = np.argmax(v)
                    model_asp_label.append('O')
                    # model_senti_label.append(classes_senti[ind_v])
                    # actual_test_senti_label.append('-1')
                    actual_test_asp_label.append('O')
                
        if M_task:
            
            paraphrase = tokenizer(text, return_tensors="pt", max_length = 250).to(device)
            label_asp = np.array(data['labels_asp']).flatten()
            size = len(paraphrase['input_ids'][0])
            print(size, paraphrase['input_ids'].shape, paraphrase['attention_mask'].shape)
            logits_asp,ls_logits_sen, loss = model(text, paraphrase['input_ids'], paraphrase['attention_mask'],\
                                                   feature, device = device, Multi_task = True)

            label_asp_senti = feature['asp_senti']
            print('>>> Sentiment for aspects')
            print('Text: ',text)

            for (aspect,senti),logits_sen in zip(label_asp_senti,ls_logits_sen):
                ind = torch.argmax(logits_sen.clone().detach()).item()
                print('label: ', aspect[0],  senti[0])
                print('predict: ',aspect[0], ind)
                actual_test_senti_label.append(int(senti[0]))
                model_senti_label.append(int(ind))
            print('='*40)
        
            n_test_total += (size-2)
            tokens = convert_ids_to_string(paraphrase['input_ids'][0], tokenizer)
            eval_data.append(tokens)

#             
            for ix,(u) in enumerate(tqdm(zip(logits_asp[0].clone().detach()), desc = f"{i}th_sentence")):
                try:
                    ind_u = torch.argmax(u[0].clone().detach()).item()
#                     
                    if (classes_asp[ind_u] == label_asp[ix-1]): n_test_asp_correct += 1
                    model_asp_label.append(classes_asp[ind_u])
                    actual_test_asp_label.append(label_asp[ix-1])


                except:
#                   
#                     ind_u = torch.argmax(u[0].clone().detach()).item()
                    
                    model_asp_label.append('O')
                    
                    actual_test_asp_label.append('O')
            
        
        
        accuracy_asp = accuracy_score(actual_test_asp_label, model_asp_label)
        accuracy_senti = accuracy_score(actual_test_senti_label, model_senti_label)

        print(f">>> accuracy aspect: {accuracy_asp}")
        print(f">>> accuracy senti: {accuracy_senti}")

        total_asp_actual.append(actual_test_asp_label)
        total_senti_actual.append(actual_test_senti_label)
        total_asp_predict.append(model_asp_label)
        total_senti_predict.append(model_senti_label)

        

        if accuracy_asp < 0.9:
            print('>>>>> Token classification <<<<<' )
            print("TOKEN - PREDICT ASP - LABEL ASP")
            print('-'*50)
            print(len(actual_test_asp_label), len(model_asp_label), size)
            # Wrong_tokens.append(tokens)
            # Wrong_aspect.append(model_asp_label)
            for ix in range(size):
                if ix == 0 or ix == size-1:
                    print(f"{tokens[ix]} \t -- {model_asp_label[ix]}   --   None")
                else:
                    print(f"{tokens[ix]} \t -- {model_asp_label[ix]}   --  {actual_test_asp_label[ix]}")
            
        print("==========================")
        




    # print(list(np.concatenate(total_asp_predict).flat))
    asp_predict, senti_predict = total_asp_predict, total_senti_predict
    total_asp_predict = list(np.concatenate(total_asp_predict).flat)
    total_senti_predict = list(np.concatenate(total_senti_predict).flat)
    total_asp_actual = list(np.concatenate(total_asp_actual).flat)
    total_senti_actual = list(np.concatenate(total_senti_actual).flat)

    print('#'*80)
    print('FINAL')
    # accuracy = n_test_correct / n_test_total
    accuracy_asp = accuracy_score(total_asp_actual, total_asp_predict)
    accuracy_senti = accuracy_score(total_senti_actual, total_senti_predict)
    # print(f"accuracy total: {accuracy}")
    print(f"accuracy aspect: {accuracy_asp}")
    print(f"accuracy senti: {accuracy_senti}")
    F1_aspect = f1_score(total_asp_actual, total_asp_predict,average='macro')
    F1_sentiment = f1_score(total_senti_actual, total_senti_predict,average='macro')
    print(f"f1_score_ASPECT: {F1_aspect}",)
    print(f"f1_score_SENTIMENT: {F1_sentiment}")
    Result['acc_asp'] =  accuracy_asp
    Result['acc_senti'] = accuracy_senti
    Result['F1_asp'] = F1_aspect
    Result['F1_senti'] = F1_sentiment
    # confusion matrix
    matrix_senti = confusion_matrix(total_senti_actual,total_senti_predict, labels = [0,1,2])
    print(f'Confusion matrix for label senti : {classes_senti} \n{matrix_senti}')
    matrix_tag = confusion_matrix(total_asp_actual,total_asp_predict, labels = classes_asp)
    print(f'Confusion matrix for label aspect : {classes_asp} \n{matrix_tag}')
    return Result, matrix_senti, matrix_tag, eval_data



In [35]:
# TO TEST THE NEWLY TRAINED MODEL
# model = model_both

# TO TEST MODEL_RECAL
model = model_recall

# n = len(val_text[:])
Result_final, matrix_senti, matrix_tag, eval_data = evaluate_model_both_task(model, loaders, device,\
                                                                                    ATE = False, APC = False, M_task = True,\
                                                                                    classes_asp = tag2id ,\
                                                                                    classes_senti = sen2id_vn)

# END

In [ ]:
print(Result_final)


In [ ]:
matrix_tag, matrix_senti